# Trabalho de Implementação

## INF2912 - Otimização Combinatória
### Prof. Marcus Vinicius Soledade Poggi de Aragão
### 2015-2

### Ciro Cavani
#### BigData / Globo.com

Algoritmos de clusterização.

## Conteúdo

Esse notebook tem o desenvolvimento e avaliação do algoritmo aproximado do P-Center (algoritmo Farthest-first traversal).

A avaliação do algoritmo é baseada em um mapeamento entre a maioria dos itens que foram atribuídos a um determinado cluster e o correspondente os valores verdadeiros gerados nesse cluster.

O P-Center teve resultados muito bons.

## Dataset

In [1]:
include("../src/clustering.jl")
import Inf2912Clustering
const Clustering = Inf2912Clustering

Inf2912Clustering

In [2]:
dataset = Clustering.dataset_tiny()
Clustering.summary(dataset)
sleep(0.2)

Number of Groups: 3
Number of Features: 16
Number of Features (group): 3
Probability of Activation: 0.8
Number of Objects (total): 100
Number of Objects per Group (min): 20
Number of Objects per Group (max): 40
Number of Objects in 1: 36
Number of Objects in 2: 38
Number of Objects in 3: 26


### P-Center - Problema de Localização de Centróides

Consiste em resolver o *P-Center* determinar os objetos representantes de cada grupo e classificar cada objeto como sendo do grupo com representante *mais próximo*

https://en.wikipedia.org/wiki/Metric_k-center

https://en.wikipedia.org/wiki/Farthest-first_traversal


In [3]:
let
    k = 3
    data = map(first, dataset.data)
    
    centers = Array(Array{Int64,1}, 0)
    i = rand(1:length(data))
    push!(centers, data[i])
    
    min_dist(v) = minimum(map(c -> norm(c - v), centers))
    max_index() = indmax(map(min_dist, data))
    
    while length(centers) < k
        i = max_index()
        push!(centers, data[i])
    end
    
    cluster(v) = indmin(map(c -> norm(c - v), centers))
    
    assignments = zeros(Int, length(data))
    for (i, v) in enumerate(data)
        assignments[i] = cluster(v)
    end
    
    assignments
end

100-element Array{Int64,1}:
 2
 3
 2
 2
 2
 1
 3
 2
 2
 2
 1
 1
 3
 ⋮
 3
 1
 3
 1
 3
 1
 3
 2
 2
 2
 2
 2

In [4]:
"Algoritmo de clusterização P-Center (algoritmo Farthest-first traversal)."
function pcenter(dataset, k)
    data = map(first, dataset.data)
    
    centers = Array(Array{Int64,1}, 0)
    i = rand(1:length(data))
    push!(centers, data[i])
    
    min_dist(v) = minimum(map(c -> norm(c - v), centers))
    max_index() = indmax(map(min_dist, data))
    
    while length(centers) < k
        i = max_index()
        push!(centers, data[i])
    end
    
    cluster(v) = indmin(map(c -> norm(c - v), centers))
    
    assignments = zeros(Int, length(data))
    for (i, v) in enumerate(data)
        assignments[i] = cluster(v)
    end
    
    assignments
end

pcenter(dataset, 3)

100-element Array{Int64,1}:
 3
 2
 1
 3
 3
 1
 1
 2
 3
 1
 3
 2
 1
 ⋮
 1
 1
 2
 2
 1
 3
 1
 3
 3
 3
 3
 3

In [5]:
import Clustering.mapping

"Algoritmo de clusterização P-Center (algoritmo Farthest-first traversal) \
aproximado para os grupos pré-definidos do dataset."
function pcenter_approx(dataset, k)
    assignments = pcenter(dataset, k)
    centermap = mapping(dataset, assignments, k)
    map(c -> centermap[c], assignments)
end

let
    k = dataset.groups
    @time prediction = pcenter_approx(dataset, k)
    Clustering.evaluation_summary(dataset, prediction; verbose=true)
    sleep(0.2)
end

  0.182553 seconds (180.38 k allocations: 8.577 MB, 3.31% gc time)
Matriz de Confusão:

[25 10 1
 8 20 10
 1 5 20]

Tamanho: 100
Acertos: 65
Erros: 35
Accuracy: 65.0%

Cluster 1

Tamanho: 36
Accuracy: 80.0%
Precision: 73.53%
Recall: 69.44%
F-score: 0.71

Acerto positivo: 25 (69.44%)
Acerto negativo: 55 (85.94%)
Falso negativo: 11 (31.43%)
Falso positivo: 9 (25.71%)

Cluster 2

Tamanho: 38
Accuracy: 67.0%
Precision: 57.14%
Recall: 52.63%
F-score: 0.55

Acerto positivo: 20 (52.63%)
Acerto negativo: 47 (75.81%)
Falso negativo: 18 (51.43%)
Falso positivo: 15 (42.86%)

Cluster 3

Tamanho: 26
Accuracy: 83.0%
Precision: 64.52%
Recall: 76.92%
F-score: 0.7

Acerto positivo: 20 (76.92%)
Acerto negativo: 63 (85.14%)
Falso negativo: 6 (17.14%)
Falso positivo: 11 (31.43%)



In [6]:
Clustering.test_dataset("small", pcenter_approx)
sleep(0.2)

  0.017872 seconds (37.53 k allocations: 11.258 MB)
Matriz de Confusão:

[356 1 2
 0 344 0
 0 0 297]

Tamanho: 1000
Acertos: 997
Erros: 3
Accuracy: 99.7%

Cluster 1

Tamanho: 359
Accuracy: 99.7%
Precision: 100.0%
Recall: 99.16%
F-score: 1.0

Acerto positivo: 356 (99.16%)
Acerto negativo: 641 (100.0%)
Falso negativo: 3 (100.0%)
Falso positivo: 0 (0.0%)

Cluster 2

Tamanho: 344
Accuracy: 99.9%
Precision: 99.71%
Recall: 100.0%
F-score: 1.0

Acerto positivo: 344 (100.0%)
Acerto negativo: 655 (99.85%)
Falso negativo: 0 (0.0%)
Falso positivo: 1 (33.33%)

Cluster 3

Tamanho: 297
Accuracy: 99.8%
Precision: 99.33%
Recall: 100.0%
F-score: 1.0

Acerto positivo: 297 (100.0%)
Acerto negativo: 701 (99.72%)
Falso negativo: 0 (0.0%)
Falso positivo: 2 (66.67%)



In [7]:
Clustering.test_dataset("large", pcenter_approx)
sleep(0.2)

  0.209626 seconds (415.55 k allocations: 113.115 MB, 37.14% gc time)
Matriz de Confusão:

[4067 8 11
 14 2878 7
 26 5 2984]

Tamanho: 10000
Acertos: 9929
Erros: 71
Accuracy: 99.29%

Cluster 1

Tamanho: 4086
Accuracy: 99.41%
Precision: 99.03%
Recall: 99.53%
F-score: 0.99

Acerto positivo: 4067 (99.53%)
Acerto negativo: 5874 (99.32%)
Falso negativo: 19 (26.76%)
Falso positivo: 40 (56.34%)

Cluster 2

Tamanho: 2899
Accuracy: 99.66%
Precision: 99.55%
Recall: 99.28%
F-score: 0.99

Acerto positivo: 2878 (99.28%)
Acerto negativo: 7088 (99.82%)
Falso negativo: 21 (29.58%)
Falso positivo: 13 (18.31%)

Cluster 3

Tamanho: 3015
Accuracy: 99.51%
Precision: 99.4%
Recall: 98.97%
F-score: 0.99

Acerto positivo: 2984 (98.97%)
Acerto negativo: 6967 (99.74%)
Falso negativo: 31 (43.66%)
Falso positivo: 18 (25.35%)

